In [1]:
import findspark
findspark.init()
import os
import time
import datetime
import pyspark.sql.functions as sf
from uuid import *
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext
from pyspark.sql.functions import when
from pyspark.sql.functions import col
from pyspark.sql.types import *
from pyspark.sql.functions import lit
from pyspark import SparkConf, SparkContext
from uuid import * 
from uuid import UUID
import time_uuid 
from pyspark.sql import Row
from pyspark.sql.functions import udf
from pyspark.sql.functions import monotonically_increasing_id
from pyspark.sql.window import Window as W

In [2]:
os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable

In [3]:
spark = SparkSession.builder.config("spark.jars.packages",'com.datastax.spark:spark-cassandra-connector_2.12:3.1.0').getOrCreate()

In [4]:
df = spark.read.format("org.apache.spark.sql.cassandra").options(table='tracking',keyspace='mydata').load()

In [5]:
df.show(5)
print(df.count())

+--------------------+----+----------+-----------+---+------------+-----+--------------------+---------------+--------------------+---+--------+----+------+----+------------+--------------------+---------+--------------------+----+--------------------+-------------------+------------+-----------+----------+----------+--------+---+--------+
|         create_time| bid|        bn|campaign_id| cd|custom_track|   de|                  dl|             dt|                  ed| ev|group_id|  id|job_id|  md|publisher_id|                  rl|       sr|                  ts|  tz|                  ua|                uid|utm_campaign|utm_content|utm_medium|utm_source|utm_term|  v|      vp|
+--------------------+----+----------+-----------+---+------------+-----+--------------------+---------------+--------------------+---+--------+----+------+----+------------+--------------------+---------+--------------------+----+--------------------+-------------------+------------+-----------+----------+--------

In [6]:
def process_timeuuid(df):
    spark_time = df.select('create_time').collect()
    normal_time = []
    for i in range(len(spark_time)):
        a = time_uuid.TimeUUID(bytes = UUID(spark_time[i][0]).bytes).get_datetime().strftime('%Y-%m-%d %H:%M:%S')
        normal_time.append(a)
    spark_timeuuid = []
    for i in range(len(spark_time)):
        spark_timeuuid.append(spark_time[i][0])
    time_data = spark.createDataFrame(zip(spark_timeuuid,normal_time),['create_time','ts'])
    result = df.join(time_data,['create_time'],'inner').drop(df.ts)
    result = result.select('create_time','ts','bid','job_id','campaign_id','custom_track','group_id','publisher_id')
    return result

In [7]:
process_df=process_timeuuid(df)

In [8]:
process_df.show()

+--------------------+-------------------+----+------+-----------+------------+--------+------------+
|         create_time|                 ts| bid|job_id|campaign_id|custom_track|group_id|publisher_id|
+--------------------+-------------------+----+------+-----------+------------+--------+------------+
|c3db7990-0260-11e...|2022-07-13 04:03:34|null|  null|       null|        null|    null|        null|
|5ffc48f0-0189-11e...|2022-07-12 02:21:45|null|  null|       null|       click|    null|        null|
|28ec5a80-fea5-11e...|2022-07-08 10:03:05|null|  null|       null|       click|    null|        null|
|a44487f0-08de-11e...|2022-07-21 10:19:45|null|  null|       null|        null|    null|        null|
|4ea14ac0-0c00-11e...|2022-07-25 09:58:17|   0|  1527|        222|        null|    null|           1|
|e0d81db0-0bfb-11e...|2022-07-25 09:26:35|   1|   188|         48|       click|      34|           1|
|40089000-0975-11e...|2022-07-22 04:17:50|null|  null|       null|       click|   